[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/179vQNSRsmT_GohvbFt_e8UucoW21pZPE?usp=sharing)

## Fine-Tuning Sentence Transformers

### **1. Install and Import Relevant Libraries**

First, we install the `sentence-transformers` library:

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

We will be utilising datasets provided by Hugging Face:

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

Note: when running the code in this article, some users may be greeted with an error about the accelerate module in Python. To fix this, run:

In [3]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.4 MB/s eta 0:00:00


Amazing! We have installed the relevant modules needed to start fine-tuning. Let’s import the necessary libraries. These will be explained as we continue through the code.

In [4]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.evaluation import TripletEvaluator

### **2. Load a Pre-Trained Model**

The powerful thing about sentence transformers is that you can take a pre-trained model and fine-tune it to adapt it to a specific task or domain, thereby significantly improving its performance on that particular application. So, we load a pre-trained model:

In [5]:
model = SentenceTransformer("microsoft/mpnet-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/532M [00:00<?, ?B/s]

Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/472k [00:00<?, ?B/s]

Let’s break this down:

- **`SentenceTransformer`**: This class provides a simple interface for using pre-trained sentence embedding models.
- **`"microsoft/mpnet-base"`**: This is a specific pre-trained model developed by Microsoft, known for its performance in various natural language processing tasks.

### **3. Loading and Preparing the Dataset**

We now need to load a dataset to fine-tune our model on. We load a dataset called "all-nli" from sentence-transformers, specifically for triplet-based training, as discussed earlier. The dataset is split into training, evaluation, and test sets.

In [6]:
dataset = load_dataset("sentence-transformers/all-nli", "triplet")
train_dataset = dataset["train"].select(range(100_000))
eval_dataset = dataset["dev"]
test_dataset = dataset["test"]

Generating train split:   0%|          | 0/557850 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6584 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6609 [00:00<?, ? examples/s]

We have the following variables:

- **`dataset`**: This variable holds the entire dataset loaded from the `"sentence-transformers/all-nli"` dataset with the `"triplet"` configuration (each example consists of triplets: anchor, positive, negative). The `load_dataset` function is used to download and prepare the dataset for use.
- **`train_dataset`**: This variable contains a subset of the training dataset. Specifically, it selects the first 100,000 samples from the "train" split of the dataset. The `select(range(100_000))` function is used to create this subset, which is typically done to limit the dataset size for faster training and experimentation.
- **`eval_dataset`**: This variable contains the evaluation dataset, also known as the validation set. It is obtained from the "dev" split of the original dataset. The evaluation dataset is used to evaluate the model's performance during training, helping to tune hyperparameters and prevent overfitting.
- **`test_dataset`**: This variable contains the test dataset, which is obtained from the "test" split of the original dataset. The test dataset is used to assess the final performance of the trained model on unseen data, providing an unbiased evaluation of its generalization capabilities.

### **4. Defining a Loss Function**

We now define the loss function used during training. For this example we use Multiple Negatives Ranking (MNR) as it’s the preferred loss function.

In [7]:
loss = MultipleNegativesRankingLoss(model)

You have the flexibility to use different loss functions if you wish to. See this [documentation](https://sbert.net/docs/sentence_transformer/loss_overview.html) for more information.

### **5. Creating a Trainer and Training the Model**

We initiate the training process by defining the following code. We have:

- **`SentenceTransformerTrainer`**: This class handles the training loop, evaluation, and optimization of the Sentence Transformer model. Note, we pass `model`, `train_dataset`, `eval_dataset` and `loss` as these are fundamental to an effective training process.
- **`train()`**: This method starts the training process.

This training process may take a couple of hours. You will see a progress bar and when computed, you will also see the **Step** and **Training Loss.** This will continue to be updated until the training is complete.

In [8]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
)
trainer.train()

Step,Training Loss
500,0.969900
1000,0.816000
1500,0.687600
2000,0.602000
2500,0.569300
3000,0.535000
3500,0.517000
4000,0.558600
4500,0.469500
5000,0.481800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.969900
1000,0.816000
1500,0.687600
2000,0.602000
2500,0.569300
3000,0.535000
3500,0.517000
4000,0.558600
4500,0.469500
5000,0.481800


TrainOutput(global_step=37500, training_loss=0.2865847708129883, metrics={'train_runtime': 8897.3008, 'train_samples_per_second': 33.718, 'train_steps_per_second': 4.215, 'total_flos': 0.0, 'train_loss': 0.2865847708129883, 'epoch': 3.0})

**Step** refers to the current iteration of the training loop. It is an indicator of how many batches have been processed. If you see "Step 10", it means the trainer has processed 10 batches of data. Each step usually corresponds to one forward and backward pass through the model with one batch of data.

**Training Loss** is a measure of how well the model is performing on the training data at each step. The loss function calculates the difference between the model's predictions and the actual target values. The objective of training is to minimize this loss.

As we can see, the training loss is decreasing as the training loops through higher iterations. This is a good sign!

### **6. Evaluating the Trained Model**

After fine-tuning, we can evaluate the model using a `TripletEvaluator` to assess its performance on the test set. This will tell us how well our fine-tuning has done.

Let’s first evaluate our *initial* pre-trained model before any fine-tuning.

Here, we define the default base model we had to begin with as `old_model`. We set up a `test_evaluator` which will assess the model’s performance on the `test_dataset`.

This will provide accuracy metrics for our model using different distance measures. If you’re unfamiliar with distance metrics, we cover them [in this article.](https://www.marqo.ai/course/introduction-to-vector-embeddings)

In [9]:
old_model = SentenceTransformer(
    "microsoft/mpnet-base",
)

test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="all-nli-test",
)
test_evaluator(old_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'all-nli-test_cosine_accuracy': 0.6594038432440611,
 'all-nli-test_dot_accuracy': 0.452262066878499,
 'all-nli-test_manhattan_accuracy': 0.7147828718414283,
 'all-nli-test_euclidean_accuracy': 0.6562263579966712,
 'all-nli-test_max_accuracy': 0.7147828718414283}

The description in the output provides us with information about the similarity metric. The value indicates that the model correctly identified the positive example over the negative example X% of the time using said similarity measure.

For example,
`'all-nli-test_cosine_accuracy': 0.6594038432440611,`
says that the model correctly identified the positive example over the negative example 65.94% of the time using cosine similarity.

These are average results, the similarity measures are generally quite low and are in need of improvement. Let’s see if our newly fine-tuned model improves these.

We set up an evaluator again but this time, on our fine-tuned model.

In [10]:
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="all-nli-test",
)
test_evaluator(model)

{'all-nli-test_cosine_accuracy': 0.908760780753518,
 'all-nli-test_dot_accuracy': 0.08987743985474353,
 'all-nli-test_manhattan_accuracy': 0.9067937660765623,
 'all-nli-test_euclidean_accuracy': 0.9073990013617794,
 'all-nli-test_max_accuracy': 0.908760780753518}

We have high accuracy values for cosine, Manhattan, and Euclidean distances which indicate that our model performs well in distinguishing between positive and negative examples in the triplet evaluation. Awesome news! The low dot product accuracy, however, suggests that it is not an effective measure in this context and can be disregarded for evaluating our model.

We can see very clearly that our fine-tuning has significantly improved the overall performance of our model. Pretty cool!

Now, it’s not always the case that the more you fine-tune your model, the better it will become. There are several quantities that affect the overall performance of your model. Head back to our article to take a look at what these are.